![](../images/rivacon_frontmark_combined_header.png)
# Equity Stochastic Volatility Models

In [ ]:
# all necessary imports and global definitions
import pyvacon.analytics as analytics
import datetime as dt
import time
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
from pylab import rcParams
import math
import copy
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot #import module for plotting functionality
import pyvacon.marketdata.converter as mkt_converter
import pyvacon.instruments.converter as ins_converter
import pyvacon.pricing.tools as pricing_tools
import pyvacon.models.plot as model_plot
import pyvacon.models.tools as model_tools
import pyvacon.models.converter as model_converter
import qgrid
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Markdown, display
import pandas as pd
import logging

%config Application.log_level="INFO"

#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline
#%matplotlib notebook

# define global variables
refdate = analytics.ptime(2017,8,14,15,30,0) #dates which enters analytics objects must be analytics ptimes.
num_sims = 40000
model_dict = {}
models = {}

calibrate_stoch_vol = False
calibrate_stoch_loc_vol = True
simulate_paths = True

# test implied volatility surface data which is used as calibration target
underlying={
    'TEST':{
            'SPOT': 100,
            'DIVIDENDS': 
                  pd.DataFrame({ 
                      'EXDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                      'PAYDATES': [dt.datetime(2018,1,1), dt.datetime(2019,1,1), dt.datetime(2020,1,1), dt.datetime(2021,1,1)],
                      'CASH': [0.0, 0.0, 0.0, 0.0],
                      'YIELD': [0.0, 0.0, 0.0, 0.0],
                      'TAX': [0.0, 0.0, 0.0, 0.0]
                  }),
            'BORROW':
              {
                'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
                'RATES' : [0.0, 0.0]
              },
            'VOLATILITY':{
                  'TYPE': 'SSVI',
                  'EXPIRIES': [10, 100, 365, 730], 
                  'ATMVOLS': [0.15, 0.17, 0.19, 0.2], 
                  'RHO': -0.65, 
                  'ETA': 0.8, 
                  'GAMMA': 0.5
                  }
              }
}

discount = {
    'EUR':{
        'DATES' : [dt.datetime(2018,1,1), dt.datetime(2020,1,1)],
        'RATES' : [0.00, 0.00]
    }
}

vol = mkt_converter.vol_from_dict(underlying, discount, 'TEST', 'EUR', refdate)

## Stochastic Volatility Models
As the name indicates in stochastic volatility models the volatility of the spot is also a stochastic process. Most of them have the form
$$ dS = \mu S dt + \sigma(S,t) S W_S $$
where $\sigma(S,t)$ is also described by a stochastic process. Since we are using the Bühler model, we assume without loss of generality $\mu = 0$ and $S(0)=1.0$.

There are a lot of different models and we will concentrate on the Heston and Scott-Chesney models:
- **Heston**
- **Scott-Chesney**
- Stein and Setin/Schöble Zhu
- Hull-White
- Hagan

In the next subsection we briefly describe the models and create them as pyvacon objects.
(See  [StochasticVolatilityModels-PastPresentAndFuture](https://jaeckel.000webhostapp.com/StochasticVolatilityModels-PastPresentAndFuture.pdf) for a discussion of all of the above models and more details)



### Heston
\begin{align}
dS = \mu S dt +\sqrt{v}SdW_S \\
dv = \kappa(\theta-v)dt+\alpha\sqrt{v}dW_v \\
<dW_s,dW_v> = \rho
\end{align}
Volatility may become zero unless the parameters are such that the (Feller) condition
\begin{align}
    \kappa\theta >\frac{1}{2}\alpha^2
\end{align}
holds true.

In [ ]:
# set Heston parameter
heston_ref =  {'OBJ_ID': 'HESTON', 
               'TYPE': 'HESTON', 
               'S0': 1.0, 
               'KAPPA': 6.602490036240081, 
               'THETA': 0.02798645471115531, 
               'ALPHA': 0.8287155390363634, 
               'V0': 0.03280615553884353, 
               'RHO': -0.7760343895576935}
model_dict['HESTON'] = heston_ref
models['HESTON']  = model_converter.stochvolmodel_from_dict(model_dict['HESTON'], refdate)

if False: #include a Heston with zero vol-of-variance to test for local volatility
    models['HESTON_FLAT_VOL'] = model_converter.stochvolmodel_from_dict({ 'OBJ_ID': 'HESTON_FLAT_VOL',
                    'TYPE': 'HESTON',
                    'S0':1.0,
                    'KAPPA':1.7961805947284033,
                    'THETA': 0.0225015745510251115,
                    'ALPHA': 0.0,
                    'V0' : 0.0225015745510251115,
                    'RHO': -0.8
                  }, refdate)

In [ ]:
# compute implied vols
expiries = [90, 180, 365]
max_num_threads = 2
xstrikes = np.arange(0.8, 1.2, 0.05)
iv = model_tools.compute_implieds(expiries, models['HESTON'], num_sims, 365, np.arange(0.8, 1.2, 0.05), refdate, max_num_threads)

In [ ]:
# compute implied vols for shifted heston
heston_new = copy.deepcopy(heston_ref)
heston_new['KAPPA'] *= 1.0
heston_new['THETA'] *= 0.50
heston_new['ALPHA'] *= 1.0
heston_new['V0'] *= 1.0
heston_new['RHO'] *= 1.0
heston_new = model_converter.stochvolmodel_from_dict(heston_new, refdate)
iv_shifted = model_tools.compute_implieds(expiries, heston_new, num_sims, 365, xstrikes, refdate, max_num_threads)

In [ ]:
# plot implieds of the two heston models
rcParams['figure.figsize'] = 20, 4
plt.subplot(1,3,1)
plt.plot(xstrikes, iv[0,:], '-x', label='unshifted')
plt.plot(xstrikes, iv_shifted[0,:], '-x', label='shifted')
plt.legend()
plt.title('90 days iv')
plt.subplot(1,3,2)
plt.plot(xstrikes, iv[1,:], '-x', label='unshifted')
plt.plot(xstrikes, iv_shifted[1,:], '-x', label='shifted')
plt.legend()
plt.title('180 days iv')
plt.subplot(1,3,3)
plt.plot(xstrikes, iv[2,:], '-x', label='unshifted')
plt.plot(xstrikes, iv_shifted[2,:], '-x', label='shifted')
plt.legend()
plt.title('1 yr iv')


### Scott-Chesney
\begin{align}
dS = \mu S dt +e^ySdW_S\\
dy = \kappa(\theta - y)dt + \alpha dW_y \\
<dW_s,dW_v> = \rho
\end{align}

In [ ]:
#setup model parameter
model_dict['SCOTT_CHESNEY'] = {'OBJ_ID': 'SCOTT_CHESNEY', 
                               'TYPE': 'SCOTT_CHESNEY', 
                               'S0': 1.0, 
                               'KAPPA': 0.08437217322463386, 
                               'THETA': -9.812444706126039, 
                               'ALPHA': 1.1505654142647475, 
                               'V0': -1.8839711373791805, 
                               'RHO': -0.7726808266170035}
models['SCOTT_CHESNEY'] = model_converter.stochvolmodel_from_dict(model_dict['SCOTT_CHESNEY'], refdate)

In [ ]:
# compute implied vols
expiries = [90, 180, 365]
max_num_threads = 2
xstrikes = np.arange(0.8, 1.2, 0.05)
iv_sc = model_tools.compute_implieds(expiries, models['SCOTT_CHESNEY'], num_sims, 365, np.arange(0.8, 1.2, 0.05), refdate, max_num_threads)

In [ ]:
# compute implied vols for shifted Scot Chesney
sc_new = copy.deepcopy(model_dict['SCOTT_CHESNEY'])
sc_new['KAPPA'] *= 7.0
sc_new['THETA'] *= 0.50
sc_new['ALPHA'] *= 3.0
sc_new['V0'] *= 1.0
sc_new['RHO'] *= 0.5
sc_new = model_converter.stochvolmodel_from_dict(sc_new, refdate)
iv_sc_shifted = model_tools.compute_implieds(expiries, sc_new, num_sims, 365, xstrikes, refdate, max_num_threads)

In [ ]:
# plot implieds of the two heston models
rcParams['figure.figsize'] = 20, 4
plt.subplot(1,3,1)
plt.plot(xstrikes, iv_sc[0,:], '-x', label='unshifted')
plt.plot(xstrikes, iv_sc_shifted[0,:], '-x', label='shifted')
plt.legend()
plt.title('90 days iv')
plt.subplot(1,3,2)
plt.plot(xstrikes, iv_sc[1,:], '-x', label='unshifted')
plt.plot(xstrikes, iv_sc_shifted[1,:], '-x', label='shifted')
plt.legend()
plt.title('180 days iv')
plt.subplot(1,3,3)
plt.plot(xstrikes, iv_sc[2,:], '-x', label='unshifted')
plt.plot(xstrikes, iv_sc_shifted[2,:], '-x', label='shifted')
plt.legend()
plt.title('1 yr iv')

### Model calibration
Each stochastic volatility model can be calibrated to given implied volatility surface. Here, for a defined set of expiries $T_j$, $1\leq j\leq M$, and X-Strikes $X_i$, $1\leq i\leq N$, call prices $C_{i,j}$ are computed and the model parameters are determined (by an optimization algorithm) such that the error function 
$$
E(x) = \sum_{i,j}(C_{i,j}-\tilde C_{i,j})^2,
$$
where $\tilde C_{i,j}$ is the model call prices computed by Monte Carlo simulation.

In [ ]:
#model calibration
model_id = 'SCOTT_CHESNEY' # SCOTT_CHESNEY HESTON
model_tools.logger.setLevel(logging.INFO)
def calibrate():
    model = models[model_id]
    strikes = [0.8, 0.9, 1.0, 1.1, 1.2]
    expiries = [90, 180, 365]
    n_sims = 8000 # number of Monte Carlo simulations used to compute the model call prices 
    n_steps_per_year = 365
    max_num_threads = 2
    calibrator = model_tools.BuehlerStochVolCalibrator(refdate, expiries, strikes, vol, model,'dummy', n_sims, n_steps_per_year, max_num_threads)

    #setup parameter mapping
    def heston_param_mapping(x,y):
        x[0] = abs(y[0])
        x[1] = abs(y[1])
        x[2] = abs(y[2])
        x[3] = abs(y[3])
        x[4] = 1.95*np.arctan(y[4])/np.pi
        
    def heston_inv_param_mapping(x,y):
        x[0] = y[0]
        x[1] = y[1]
        x[2] = y[2]
        x[3] = y[3]
        x[4] = np.tan(y[4]*np.pi/1.95)

    def scott_chesney_mapping(x,y):
        x[0] = y[0]
        x[1] = abs(y[1])
        x[2] = abs(y[2])
        x[3] = y[3]
        x[4] = 1.95*np.arctan(y[4])/np.pi

    def scott_chesney_inv_mapping(x,y):
        x[0] = y[0]
        x[1] = abs(y[1])
        x[2] = abs(y[2])
        x[3] = y[3]
        x[4] = np.tan(y[4]*np.pi/1.95)

    param_mappings = {
        'HESTON' :  heston_param_mapping,
        'SCOTT_CHESNEY' : scott_chesney_mapping
    }  
    inv_param_mappings = {
        'HESTON' :  heston_inv_param_mapping,
        'SCOTT_CHESNEY' : scott_chesney_inv_mapping
    }  

    #calibrate
    startvalues = analytics.vectorDouble()
    model.getParameters(startvalues)
    optim_options = {'xtol': 1e-5, 'ftol': 1e-5, 'maxiter': 150, 'maxfev': 1500, 'disp': True}
    
    print(model_dict[model_id])
    inv_param_mappings[model_id](startvalues, startvalues)
    calibrator.calibrate(startvalues, param_mappings[model_id], 'Nelder-Mead', optim_options) # Powell, Nelder-Mead, BFGS
    model.getParameters(startvalues)
    model_converter.stochvolmodel_to_dict(model, model_dict[model_id])
    print(model_dict[model_id])
    
    return calibrator
    #output calibrated parameters
if False: calibrator = calibrate()

In [ ]:
#plot calibration errors
if False : calibrator.plot_error_vols( [0,1,2], True )

## Stochastic Local Volatility
$$ dS/S = \mu dt + \sigma(S,t)vdW_t,$$
$$ dv = \nu(v,t) dt + \sigma^v(v,t) dW^v_t$$
where $\sigma(S,t)$ is the stochastic-local-volatility calibrated so that the model perfectly fits a given volatility surface.

The stochastic-local volatility may be derived from the leverage function $l(S,t)= E\left(v^2\mid S_t=S\right)$ and the local volatility $\sigma^{lv}(S,t)$ by
$$
\sigma(S,t) = \sigma^{lv}(S,t) = \frac{\sigma^{lv}(S,t)}{\sqrt{l(S,t)}}
$$

### Calibrating the stochastic local volatility function
- Time discretization $t_i$, $1\leq i \leq N$
- $l(S_0,0) = \frac{\sigma^{lv}(S_0,0)}{v(0)}$

- Use Euler discretized MC with regression to derive leverage function/local vol of process
    - $ v(t_{i+1}) = \nu(v(t_i,t_i))(t_{i+1}-t_i) + \sigma^v(v(t_i),t_i)\varepsilon_i $
    - $S(t_{i+1}) = S(t_i)\mu(t_{i+1}-t_i) + \sigma(S(t_i), t_i)v(t_i)\tilde \varepsilon_i$
- Use simulated values $S(t_{i+1})$ and $ v(t_{i+1})$ to estimate 
$$l\left(S(t_{i+1}),t_{i+1}\right)= E\left(v(t_{i+1})^2\mid S_{t_{i+1}}=S_{t_i}\right)$$
via regression.
- Also possible to calibrate to a stochastic volatility process of higher dimensions

Alternative: PDE, but numerically much more difficult and not very generic, see [here](https://www.rivacon.com/wp-content/uploads/2017/09/stoch_local_vol.pdf) for an example for the Heston model.

In [ ]:
#calibrate stochastic local volatility
if calibrate_stoch_loc_vol:
    time_grid = analytics.vectorDouble(pl.frange(1.0/365.0, 3.0, 1.0/365.0))
    stoch_local_vol ={}
    def get_regression_parameter():
        rbf_param =  analytics.RBFRegressionParameter()
        rbf_param.nCenters = 25
        rbf_param.includeLinearFunctions = False
        rbf_param.scalingFactor = 0.01

        polynom_param = analytics.PolynomialRegression1DParameter('')
        polynom_param.degree = 20

        pcw_linear_param = analytics.PiecewiseLinearRegression1DParameter('')
        pcw_linear_param.nGridPoints = 80
        pcw_linear_param.smoothnessPenalty = 0.000001
        
        return pcw_linear_param

    def calibrate_stoch_loc_vol():
        #analytics.setLogLevel('INFO')
        leverage_calib_param = analytics.StochLocalVolFunctionCalibratorParameter()
        leverage_calib_param.pgParam = analytics.PathGeneratorParameter()
        leverage_calib_param.pgParam.numberOfSimulations = 200000
        leverage_calib_param.regressionParam = get_regression_parameter()
        return analytics.StochLocalVolFunctionCalibrator.calibrate(refdate, leverage_calib_param, time_grid, stoch_vol_model, vol)
    
    for model in ['HESTON', 'SCOTT_CHESNEY']: #
        stoch_vol_model = models[model]
        stoch_local_vol[model] = calibrate_stoch_loc_vol()
        models[model + '_LV'] = analytics.StochasticLocalVolatility(models[model], stoch_local_vol[model])

In [ ]:
#plot stochastic local volatility
def plot_stoch_loc_vol(time_slices, spot_slices, stoch_local_vol):
    rcParams['figure.figsize'] = 12, 5
    spot_grid = analytics.vectorDouble(pl.frange(0.50, 1.5,0.01))
    lev_surf = stoch_local_vol.eval(time_grid, spot_grid)
    plt.subplot(1,2,1)
    for i in range(len(time_slices)):
        plt.plot(spot_grid,lev_surf[time_slices[i]], '-x', label='ttm %.2f' % time_grid[time_slices[i]])
        plt.xlabel('X')
        plt.ylabel('stochastic local volatility')
    plt.legend()
    plt.title('spot slices')
    plt.subplot(1,2,2)
    for i in range(len(spot_slices)):
        timeline = []
        for j in range(len(time_grid)):
            
            timeline.append(lev_surf[j][spot_slices[i]])
        plt.plot(time_grid, timeline, '-o', label = 'spot ' + str(spot_grid[spot_slices[i]]))
        plt.xlabel('t')
        plt.ylabel('stochastic local volatility')
    plt.title('time slices')
  
    plt.legend()

plot_stoch_loc_vol([50, 180, 365], [0, 50, 99], stoch_local_vol['SCOTT_CHESNEY']) # 'SCOTT_CHESNEY' HESTON

## Simulating Paths
All tools to analyze the models are based on simulated paths. Therefore we first have to simulate the model of interest.

In [ ]:
# simulate paths for all models using the analytics.ModelLab
model_labs = {}
simtimes_p = range(1,3*365)
simtimes = converter.createPTimeList(refdate, simtimes_p)
max_num_threads = 2
def simulate_paths():
    ntimesteps_per_year = 365
    for key, model in models.items():
        tmp = analytics.ModelLab(model, refdate)
        tmp.simulate(simtimes, num_sims, ntimesteps_per_year, max_num_threads) 
        model_labs[key] = tmp
start_time = time.time()
if simulate_paths: simulate_paths()
print("--- %s seconds ---" % (time.time() - start_time))

## Calibration Errors

In [ ]:
# calibration errors
def plot_implieds_expiry(expiry_index, stoch_vol_model):
    strikes =  pl.frange(0.8,1.20, 0.020)
    vols = []
    vols_stoch_vol = []
    call_prices = model_tools.compute_statistics(model_labs[stoch_vol_model + '_LV'], strikes, expiry_index, 0, lambda x, y: max(x-y,0.0))
    call_prices_stoch_vol = model_tools.compute_statistics(model_labs[stoch_vol_model], strikes, expiry_index, 0, lambda x, y: max(x-y,0.0))
    dc = analytics.DayCounter(enums.DayCounter.ACT365_FIXED)
    expiry_yf = dc.yf(refdate, simtimes[expiry_index])
    for i in range(len(call_prices)):
        vols.append(analytics.calcImpliedVol(call_prices[i], strikes[i], expiry_yf, 1.0, 1.0, 'C'))
        vols_stoch_vol.append(analytics.calcImpliedVol(call_prices_stoch_vol[i], strikes[i], expiry_yf, 1.0, 1.0, 'C'))
    vol_ref = []
    for i in strikes:
        vol_ref.append(vol.calcImpliedVol(refdate, simtimes[expiry_index], i))
    plt.plot(strikes, vol_ref, '-o', label='reference')
    plt.plot(strikes, vols_stoch_vol, '-x', label=stoch_vol_model)
    plt.plot(strikes, vols, '-x', label=stoch_vol_model+'_LV')
    plt.title('expiry ' + str(expiry_index))
    plt.legend()
    
def plot_implieds_expiries():
    plt.subplot(2,4,1)
    plot_implieds_expiry(180, 'HESTON')
    plt.subplot(2,4,2)
    plot_implieds_expiry(365, 'HESTON')
    plt.subplot(2,4,3)
    plot_implieds_expiry(365+180, 'HESTON')
    plt.subplot(2,4,4)
    plot_implieds_expiry(2*365, 'HESTON')
    plt.subplot(2,4,5)
    plot_implieds_expiry(180, 'SCOTT_CHESNEY')
    plt.subplot(2,4,6)
    plot_implieds_expiry(365, 'SCOTT_CHESNEY')
    plt.subplot(2,4,7)
    plot_implieds_expiry(365+180, 'SCOTT_CHESNEY')
    plt.subplot(2,4,8)
    plot_implieds_expiry(2*365, 'SCOTT_CHESNEY')

rcParams['figure.figsize'] = 24, 8
plot_implieds_expiries()

## Volatility and Variance Options
### Variance options


In [ ]:
# price variance and volatility options
var_call_prices = {}
var_strikes = pl.frange(0.7, 1.3, 0.05)
rlzd_var = {}

def price_var_option_implieds(strikes,  n_time):
    result_var = {}
    result_vol = {}
    for key, model_lab in model_labs.items():
        sims, sims_vol = model_tools.compute_realized_vol_var(model_lab, n_time)
        fwd = np.mean(sims)
        fwd_vol = np.mean(sims_vol)
        #print(fwd)
        vols = []
        vols_vol = []
        for i in range(len(strikes)):
            try:
                vols.append(analytics.calcImpliedVol(np.mean([max(x-strikes[i]*fwd,0.0) for x in sims]), strikes[i]*fwd, n_time/365.0, 1.0, fwd, 'C'))
            except RuntimeError:
                vols.append(0)
            try:
                vols_vol.append(analytics.calcImpliedVol(np.mean([max(x-strikes[i]*fwd_vol,0.0) for x in sims_vol]), strikes[i]*fwd_vol, n_time/365.0, 1.0, fwd_vol, 'C'))
            except:
                vols_vol.append(0)
        result_var[key] = vols
        result_vol[key] = vols_vol
    return result_var, result_vol
n_time = 300
var_call_vols, vol_call_vols = price_var_option_implieds(var_strikes, n_time)

In [ ]:
# plot implied vols of Call options on variance and volatility
plt.subplot(1,2,1)
rcParams['figure.figsize'] = 16, 8
for model in ['HESTON',  'SCOTT_CHESNEY', 'HESTON_LV', 'SCOTT_CHESNEY_LV']:
    plt.plot(var_strikes, var_call_vols[model], '-x', label = model)
plt.xlabel('relative (to fwd) strikes')
plt.ylabel('implied vol of call option on variance')
plt.legend()
plt.title('Calls on realized variance')
plt.subplot(1,2,2)
for model in ['HESTON',  'SCOTT_CHESNEY', 'HESTON_LV', 'SCOTT_CHESNEY_LV']:
    plt.plot(var_strikes, vol_call_vols[model], '-x', label = model)
plt.xlabel('relative (to fwd) strikes')
plt.ylabel('implied vol of call option on vol')
plt.legend()
plt.title('Calls on realized volatility')

## Forward Start Options
The payoff of forward start options is fixed at a future time (here typically the strike is fixed w.r.t. a spot at a certain time).
### Forward Start Call
European Call where the strike is fixed relative to a future spot, i.e.
$$ V(S_T) = \max(S_T-k\cdot S_{T_0},0) $$

In [ ]:
# compute prices
def price_fwd_start_call(model_lab, timeindex_fixing, timeindex_expiry, strikes):
    prev_spots = analytics.vectorDouble()
    curr_spots = analytics.vectorDouble()
    model_lab.getTimeSlice(prev_spots, timeindex_fixing, 0)
    model_lab.getTimeSlice(curr_spots, timeindex_expiry, 0)
    result = []
    for strike in strikes:
        prices = np.zeros(prev_spots.size())
        for j in range(len(prev_spots)):
            prices[j] += max(curr_spots[j] - strike*prev_spots[j], 0.0)
        result.append(np.mean(prices))
    return result

strikes = pl.frange(0.7,1.3, 0.0125)
t_fixing = 365
t_expiry = 365 + 365
fwd_start_call_prices = {}
for model in models:
    fwd_start_call_prices[model] = price_fwd_start_call(model_labs[model], t_fixing, t_expiry, strikes)

In [ ]:
# plot prices
rcParams['figure.figsize'] = 12, 8
for model in ['HESTON', 'HESTON_LV', 'SCOTT_CHESNEY', 'SCOTT_CHESNEY_LV']:
    plt.plot(strikes, fwd_start_call_prices[model], '-x', label = model)
plt.legend()

### Forward Start Call Spread

In [ ]:
# plot prices
rcParams['figure.figsize'] = 12, 8
for model in ['HESTON', 'HESTON_LV', 'SCOTT_CHESNEY', 'SCOTT_CHESNEY_LV']:
    tmp = []
    for i in range(len(strikes)-1):
        tmp.append(fwd_start_call_prices[model][i]- fwd_start_call_prices[model][i+1])
    plt.plot(strikes[0:len(strikes)-1], tmp , '-x', label = model)
plt.legend()

### Forward Start Digital Call
$$ V(S_{T}) = \left\{ \begin{array}{l} 1 \mbox{ if } \frac{S_{T}}{S_{T_0}} > K_{rel} \\
                            0 \mbox{ otherwise } \end{array} \right.$$

In [ ]:
# compute prices
def price_fwd_start_digital(model_lab, timeindex_fixing, timeindex_expiry, strikes):
    prev_spots = analytics.vectorDouble()
    curr_spots = analytics.vectorDouble()
    model_lab.getTimeSlice(prev_spots, timeindex_fixing, 0)
    model_lab.getTimeSlice(curr_spots, timeindex_expiry, 0)
    result = []
    for strike in strikes:
        prices = np.zeros(prev_spots.size())
        for j in range(len(prev_spots)):
                if curr_spots[j] > strike*prev_spots[j]:
                    prices[j] += 1.0
        prices = np.maximum(prices, 0.0)
        result.append(np.mean(prices))
    return result

strikes = pl.frange(0.7,1.3, 0.0125)
t_fixing = 365
t_expiry = 365 + 365
fwd_start_digital_prices = {}
for model in models:
    fwd_start_digital_prices[model] = price_fwd_start_digital(model_labs[model], t_fixing, t_expiry, strikes)

In [ ]:
# plot prices
rcParams['figure.figsize'] = 12, 8
for model in ['HESTON', 'HESTON_LV', 'SCOTT_CHESNEY', 'SCOTT_CHESNEY_LV']:
    plt.plot(strikes, fwd_start_digital_prices[model], '-x', label = model)

plt.legend()

## Cliquets
Cliquet options payoff is a function of relative returns.
Different types are (the interest reader is referred to [here](https://mathfinance.com/wp-content/uploads/2017/06/20100822-kilin-nalholm-wystup-costOfVol.pdf) for a detailed discussion and prices for different cliquets):
- Reverse Cliquets
- Napoleon
- Accumulator
- Call Spread Cliquet

### Reverse Cliquet
The reverse cliquet pays
$$ P =  \max \left( C+ \sum_{i=0}^{N-1} r_i^-, 0\right)$$
where $ r_i = \frac{S_{T_{i+1}}-S_{T_i}}{S_{T_i}} $ and $r_i^-=\min(r_i,0)$.

In [ ]:
#compute prices
def price_reverse_cliquet(model_lab, max_coupons, num_days_period, num_days_expiry):
    prev_spots = analytics.vectorDouble()
    curr_spots = analytics.vectorDouble()
    model_lab.getTimeSlice(prev_spots, 0, 0)
    result = []
    prices = np.zeros(prev_spots.size())
    for i in range(num_days_period,num_days_expiry,num_days_period):
        model_lab.getTimeSlice(curr_spots, i, 0)
        for j in range(len(prev_spots)):
            prices[j] += min((curr_spots[j]-prev_spots[j])/prev_spots[j], 0)
        prev_spots.swap(curr_spots)
    for C in max_coupons:
        #tmp = np.maximum(prices + C, 0.0)
        result.append(np.mean(np.maximum(prices + C, 0.0)))
    return result

max_coupons = pl.frange(0.1,0.6, 0.1)

reverse_cliquet_prices = {}
for model in models:
    reverse_cliquet_prices[model] = price_reverse_cliquet(model_labs[model], max_coupons, 30, 365)

In [ ]:
# plot prices
rcParams['figure.figsize'] = 8, 6
for model in ['HESTON', 'HESTON_LV', 'SCOTT_CHESNEY', 'SCOTT_CHESNEY_LV']:
    plt.plot(max_coupons, reverse_cliquet_prices[model], '-x', label = model)
plt.xlabel('C')
plt.ylabel('price')
plt.legend()

### Napoleon
The payoff is a combo of payoffs of the form
$$ P =  \max \left( C+ \min_i r_i, 0\right)$$
where $ r_i = \frac{S_{T_{i+1}}-S_{T_i}}{S_{T_i}} $.

In [ ]:
#compute prices
def price_simple_napoleon(model_lab, coupons, num_days_period, num_days_expiry):
    prev_spots = analytics.vectorDouble()
    curr_spots = analytics.vectorDouble()
    model_lab.getTimeSlice(prev_spots, 0, 0)
    result = []
    prices = np.full(prev_spots.size(), 10000000.1)
    for i in range(num_days_period,num_days_expiry,num_days_period):
        model_lab.getTimeSlice(curr_spots, i, 0)
        for j in range(len(prev_spots)):
            prices[j] = min((curr_spots[j]-prev_spots[j])/prev_spots[j], prices[j])
        prev_spots.swap(curr_spots)
    for C in coupons:    
        tmp = np.maximum(prices+C, 0.0)
        result.append(np.mean(tmp))
    return result

coupons = pl.frange(0.04,0.12, 0.01)

napoleon_prices = {}
for model in models:
    napoleon_prices[model] = price_simple_napoleon(model_labs[model], coupons, 30, 365)

In [ ]:
# plot prices
rcParams['figure.figsize'] = 12, 8
for model in ['HESTON', 'HESTON_LV', 'SCOTT_CHESNEY', 'SCOTT_CHESNEY_LV']:
    plt.plot(coupons, napoleon_prices[model], '-x', label = model)

plt.legend()

### Accumulator
Payoff is given by
$$ P = \max\left(0, \sum_{i=0}^{N-1}\max\left(\min(r_i, \mbox{cap}), \mbox{floor}  \right) \right)$$
where $ r_i = \frac{S_{T_{i+1}}-S_{T_i}}{S_{T_i}} $.

In [ ]:
#compute prices
def price_accumulator(model_lab, cap, floor, num_days_period, num_days_expiry):
    prev_spots = analytics.vectorDouble()
    curr_spots = analytics.vectorDouble()
    model_lab.getTimeSlice(prev_spots, 0, 0)
    result = []
    prices = np.full(prev_spots.size(), 0.0)
    for i in range(num_days_period,num_days_expiry,num_days_period):
        model_lab.getTimeSlice(curr_spots, i, 0)
        for j in range(len(prev_spots)):
            prices[j] += max(min((curr_spots[j]-prev_spots[j])/prev_spots[j], cap), floor)
        prev_spots.swap(curr_spots)
    tmp = np.maximum(prices, 0.0)
    return np.mean(tmp)

floor = -0.01
caps = pl.frange(0.006,0.02, 0.002)

accumulator_prices = {}
for model in models:
    accumulator_prices[model] = []
    for cap in caps:
        accumulator_prices[model].append(price_accumulator(model_labs[model], cap, floor, 30, 365))

In [ ]:
# plot prices
rcParams['figure.figsize'] = 12, 8
for model in ['HESTON', 'HESTON_LV', 'SCOTT_CHESNEY', 'SCOTT_CHESNEY_LV']:
    plt.plot(caps, accumulator_prices[model], '-x', label = model)

plt.legend()

### Globally capped, locally floored Cliquets
-monthly resetting, locally capped, globally floored Call
-Spot=100 ->Annahme preis < 1% bei local vol, stoch vol teurer > 2%

$\max(\sum_{i=1}^{12} (C_{S(i-1)}-P_{S(i-1)}-C_{1.04*S(i-1)}),0)$

In [ ]:
#price
def price_cliquet(model_lab, cap=0.04):
    prev_spots = analytics.vectorDouble()
    curr_spots = analytics.vectorDouble()
    model_lab.getTimeSlice(prev_spots, 0, 0)
    prices = np.zeros(prev_spots.size())
    for i in range(30,361,30):
        model_lab.getTimeSlice(curr_spots, i, 0)
        for j in range(len(prev_spots)):
            prices[j] += min(curr_spots[j]-prev_spots[j], cap*prev_spots[j])
        prev_spots.swap(curr_spots)
    prices = np.maximum(prices, 0.0)
    return np.mean(prices)
    
cliquet_prices = {}
for model in models:
    cliquet_fixing_index = 120
    cliquet_prices[model] = price_cliquet(model_labs[model])
    
print(str(cliquet_prices))

### Plotting tools

#### Plot single paths

In [ ]:
model_plot.paths(model_labs['SCOTT_CHESNEY'], range(10,20),0)

In [ ]:
model_lab = model_labs['SCOTT_CHESNEY'] # SCOTT_CHESNEY
ntimepoint = 5*300
spot = analytics.vectorDouble()
variance = analytics.vectorDouble()
for ntimepoint in [180, 360]:
    model_lab.getTimeSlice(spot, ntimepoint, 0)
    model_lab.getTimeSlice(variance, ntimepoint, 1)
    for i in range(len(variance)):
        variance[i] = math.exp(2.0*variance[i])
    plt.plot(spot, variance,'x')

## Bühler Model
So far we assumed that $S(0) = 1.0$ and $\mu = 0$. All results carry simply over to more realistic cases including cash dividends using the Bühler approach. Here, the spot is simply modeled by a so-called X-Process
The volatility surfaces provided by the analytics library are parametrized w.r.t. the so-called X-strikes, i.e. one has to put in a strike w.r.t. the X-variable which is the driving process of the spot $S$, i.e.
$$ S_t=(F_t-D_t)X_t+D_t$$
where $F_t$ is the risky forward and $D_t$ the cash dividends, see [buehler](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1141877) for a more detailed discussion. 